# Accessing Application

In [ ]:
from keras.models import load_model  
import cv2  
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import time
import pyautogui


rock=0
thumpsup=0
thumpsdown=0
peace=0
ok=0



np.set_printoptions(suppress=True)

model = load_model("model/keras_Model.h5", compile=False)

class_names = open("model/labels.txt", "r").readlines()

camera = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8, maxHands=1)

while True:

    ret, image = camera.read()
    frame=image
    hands = detector.findHands(image, draw=False)
    if len(hands)!=0:
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

        image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

        image = (image / 127.5) - 1

        prediction = model.predict(image)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]


        if confidence_score>0.90:
            cv2.putText(frame, class_name[2:], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)


            if str(class_name[0])=='1':    # ok
                ok+=1

            if class_name[0]=='0':   # rock
                rock+=1


            if class_name[0]=='2':   # Peace
                peace+=1


            if class_name[0]=='4':   # thumpsup
                thumpsup+=1


            if class_name[0]=='3':   # thumpsdown
                thumpsdown+=1


    ############################################################################################################################# ######################################################################################################################

            print(ok)
            if class_name[0]=='1' and ok>=5:
                pyautogui.press('f5')
                time.sleep(2)
                rock,thumpsup,thumpsdown,peace,ok=0,0,0,0,0

            if class_name[0]=='4' and thumpsup>=5:
                pyautogui.press('right')
                time.sleep(2)
                rock,thumpsup,thumpsdown,peace,ok=0,0,0,0,0

            if class_name[0]=='3' and thumpsdown>=5:
                pyautogui.press('left')
                time.sleep(2)
                rock,thumpsup,thumpsdown,peace,ok=0,0,0,0,0

            if class_name[0]=='2' and peace>=5:
                pyautogui.press('esc')
                time.sleep(2)
                rock,thumpsup,thumpsdown,peace,ok=0,0,0,0,0













    cv2.imshow("Webcam Image", frame)

    keyboard_input = cv2.waitKey(1)

    if keyboard_input == 27:
        break

camera.release()
cv2.destroyAllWindows()